# Geocoding Real Estate Listings

Code by Matt DeVay

In [1]:
# so the first thing is to open the gis dataset:

import pandas as pd
import numpy as np
import requests
import string

## Here we'll import the data from the webscraping to a pandas datatable

In [2]:

adverts_df = pd.read_csv('./nairobi_all_listings.csv', index_col=False)
adverts_df.head()

,id,category,list_type,beds,baths,sq_ft_price,price_KES,district,neighborhood,street
0,3394167,Flat & Apartment,For Rent,2,None,None,120000,Westlands,Brookside,None
1,3399035,Flat & Apartment,For Rent,2,None,77,96000,Ridgeways,None,None
2,3433894,Flat & Apartment,For Rent,2,None,46,70000,Dagoretti North,Valley Arcade,Lavington
3,3433821,Flat & Apartment,For Rent,2,2,None,200000,Westlands,Westlands Area,Kingfisher Nest
4,3433777,Flat & Apartment,For Rent,4,None,None,110000,Westlands,Parklands,1St Parklands


In [4]:
adverts_df.shape

(4000, 10)

## Now we can import our API key and create the base URL for API calls

In [5]:
with open('../../../google_api.txt') as key:
    api_key = '&key=' + key.read()

base_url = 'https://maps.googleapis.com/maps/api/geocode/json?address='

In [6]:
# Create the new columns we need, and replace NaN and None to 0


adverts_df['latitude'] = 0
adverts_df['longitude'] = 0
adverts_df['clean_grab'] = 0
adverts_df.replace('None', 0, inplace=True)
adverts_df.fillna(0, inplace=True)

In [7]:
# lets create a function that will return the lat and long of a row of the data frame

def get_lat_long(row):
    lat = 0
    long = 0
    if row['street']:
        address = "+".join([row['street'], row['neighborhood'], row['district'], "Nairobi"])
    elif row['neighborhood']:
        address = "+".join([row['neighborhood'], row['district'], "Nairobi"])
    else:
        address = row['district'] + "+Nairobi"
    url = base_url + address + api_key
    res = requests.get(url)
    if res.json()['status'].lower() == "ok":
        df = pd.DataFrame(res.json()['results'])
        geo = pd.DataFrame(df.loc[0, 'geometry'])
        long = geo.loc["lng", "location"]
        lat = geo.loc["lat", "location"]  
    clean = res.json()['status']
    return lat, long, clean

In [8]:
# and test it:

lat, long, clean = get_lat_long(adverts_df.loc[313,:])
lat, long, clean


(-1.278937, 36.7777984, 'OK')

In [9]:
adverts_df.head()

,id,category,list_type,beds,baths,sq_ft_price,price_KES,district,neighborhood,street,latitude,longitude,clean_grab
0,3394167,Flat & Apartment,For Rent,2,0,0,120000,Westlands,Brookside,0,0,0,0
1,3399035,Flat & Apartment,For Rent,2,0,77,96000,Ridgeways,0,0,0,0,0
2,3433894,Flat & Apartment,For Rent,2,0,46,70000,Dagoretti North,Valley Arcade,Lavington,0,0,0
3,3433821,Flat & Apartment,For Rent,2,2,0,200000,Westlands,Westlands Area,Kingfisher Nest,0,0,0
4,3433777,Flat & Apartment,For Rent,4,0,0,110000,Westlands,Parklands,1St Parklands,0,0,0


## Now lets define a function that allows us to grab results from Google and report whether or not the API call was successful, and if so, whether or not it actually grabbed a result.

In [10]:
def add_lat_long(df):
    # initiate some counters so that we can troubleshoot later
    success_count = 0
    prev_success_count = 0
    no_results_count = 0
    no_results_list = []
    fail_count = 0
    fail_list = []
    
    # loop through the dataframe 
    for i in range(len(df)): 
        
        # if there is a 0 in the clean_grab column:
        if not df.loc[i, 'clean_grab']:
            try:
                # get the lat/long from google
                lat, long, clean = get_lat_long(df.loc[i,:])
                
                # if there is a lat result, assign the lat, long, and status to the record:
                if lat:
                    df.loc[i, 'latitude'] = lat
                    df.loc[i, 'longitude'] = long
                    df.loc[i, 'clean_grab'] = clean
                    success_count +=1
                
                # if not, 
                else:
                    df.loc[i, 'clean_grab'] = clean
                    no_results_count +=1
                    no_results_list.append(i)
            except:
                df.loc[i, 'clean_grab'] = "!"
                fail_count +=1
                fail_list.append(i)
        elif df.loc[i, 'clean_grab'] == 'OK':
            prev_success_count +=1
        elif df.loc[i, 'clean_grab'] == "ZERO_RESULTS":
            no_results_count +=1
            no_results_list.append(i)
        else:
            df.loc[i, 'clean_grab'] = "!"
            fail_count +=1
            fail_list.append(i)
    
    
    print(f"{success_count} entries were geocoded successfully. \n")
    print(f"{prev_success_count} entries had already been successfully geocoded. \n")
    print(f"{no_results_count} entries returned no results from the API. \n Their indices: \n {no_results_list}")
    print(f"{fail_count} entries could not be geocoded at all.")
    print(f"The indices of the failed searches: \n {fail_list}")
    double_check_list = no_results_list + fail_list
    return double_check_list

## Now we can actually get the data!

In [ ]:
double_check = add_lat_long(adverts_df)
double_check

In [ ]:
# there were only 7 entries that google could not get a result for, and none that
# failed on syntax

adverts_df.head()

In [ ]:
# the seven rows that had no results from google. We can manually search, drop, or impute as desired

adverts_df[adverts_df['clean_grab'] != "OK"]

## Finally, we can export to CSV for further processing:

In [ ]:
# now let's write to csv for safe-keeping!
adverts_df.to_csv('./adverts_geocoded.csv')